In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[10],8
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,0
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,"[0, -1, 0]"
ng_refs,-,"[3, 2, 3]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.201621                    1 -0.160375
0.000750        2 -0.193749                    2 -0.154487
0.001052        3 -0.173303                    3 -0.140905
0.001476        4 -0.150291                    4 -0.125382
0.002070        5 -0.124865                    5 -0.107729
0.002904        6 -0.096935                    6 -0.087885
0.004074        7 -0.066493                    7 -0.065656
0.005714        8 -0.033553                    8 -0.040666
0.008015        9  0.001777                    9 -0.013026
0.011243       10  0.039302                   10  0.017406
0.015771       11  0.078720                   11  0.050732
0.022122       12  0.119697                   12  0.086950
0.031031       13  0.161652                   13  0.125745
0.043528       14  0.195934                   14  0.160852
0.061057       15  0.213425                   15  0.185521
0.085645       16  0.220409                   16  0.203092
0.120136       17  0.223983                   17  0.217281
0.168516       18  0.225183                   18  0.227600
0.236378       19  0.225961                   19  0.234137
0.331549       20  0.230585                   20  0.240069
0.465100       21  0.245497                   21  0.248288
0.652400       22  0.206924                   22  0.204572
0.915100       23  0.117138                   23  0.112281
1.283650       24  0.057291                   24  0.052235
1.800600       25  0.029391                   25  0.029110
2.525700       26  0.016002                   26  0.019200
3.542800       27  0.008655                   27  0.011618
4.969550       28  0.006010                   28  0.007126
6.970850       29  0.006509                   29  0.006491
9.778100       30  0.006719                   30  0.006775
13.715850      31  0.005864                   31  0.006582
19.239350      32  0.004319                   32  0.005322
26.987250      33  0.003964                   33  0.004965
37.855300      34  0.005127                   34  0.006313
53.100050      35  0.006135                   35  0.007312
73.887500      36  0.007056                   36  0.007533
97.662500      37  0.007832                   37  0.007577
121.437500     38  0.007723                   38  0.007360
145.212500     39  0.009336                   39  0.009501
168.987500     40  0.010314                   40  0.012049
192.762500     41  0.010586                   41  0.014187
216.537500     42  0.009396                   42  0.013884
240.312500     43  0.007327                   43  0.011320
264.087500     44  0.003851                   44  0.006120
287.862500     45 -0.002597                   45 -0.002999
311.637500     46 -0.006069                   46 -0.008910
335.412500     47  0.000037                   47 -0.004043
359.187500     48  0.010978                   48  0.006813
382.962500     49  0.026120                   49  0.019407
406.737500     50  0.044861                   50  0.032979
430.512500     51  0.062143                   51  0.050035
454.287500     52  0.076478                   52  0.069940
478.062500     53  0.086983                   53  0.086826
501.837500     54  0.093528                   54  0.094566
525.612500     55  0.096452                   55  0.095857
549.387500     56  0.096344                   56  0.095036
573.162500     57  0.094025                   57  0.095690
596.937500     58  0.090111                   58  0.097304
620.712500     59  0.085063                   59  0.095588
644.487500     60  0.079200                   60  0.092949
668.262500     61  0.072800                   61  0.085069
692.037500     62  0.066184                   62  0.075015
715.812500     63  0.059721                   63  0.064422
739.587500     64  0.053961                   64  0.054974
763.362500     65  0.049574                   65  0.048022
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug       fldg     fnetg       flug          fldg   
pressure    level                                                            
0.000000    1      -4.883807   0.000000 -4.883807  -4.888516 -1.235502e-08   
0.000624    2      -4.883828   0.000006 -4.883822  -4.888531  3.656527e-06   
0.000876    3      -4.883836   0.000008 -4.883828  -4.888537  5.231761e-06   
0.001229    4      -4.883847   0.000012 -4.883835  -4.888546  7.749819e-06   
0.001723    5      -4.883862   0.000018 -4.883844  -4.888557  1.177521e-05   
0.002417    6      -4.883881   0.000027 -4.883854  -4.888573  1.820198e-05   
0.003391    7      -4.883907   0.000042 -4.883865  -4.888593  2.844174e-05   
0.004757    8      -4.883942   0.000065 -4.883876  -4.888620  4.469187e-05   
0.006672    9      -4.883986   0.000102 -4.883884  -4.888655  7.042197e-05   
0.009359    10     -4.884042   0.000159 -4.883883  -4.888700  1.110498e-04   
0.013128    11     -4.884113   0.000247 -4.883866  -4.888756  1.749986e-04   
0.018415    12     -4.884198   0.000382 -4.883816  -4.888824  2.752701e-04   
0.025830    13     -4.884297   0.000586 -4.883711  -4.888904  4.317254e-04   
0.036232    14     -4.884405   0.000893 -4.883512  -4.888992  6.742107e-04   
0.050823    15     -4.884515   0.001341 -4.883173  -4.889081  1.041753e-03   
0.071291    16     -4.884623   0.001968 -4.882656  -4.889169  1.579679e-03   
0.100000    17     -4.884724   0.002818 -4.881906  -4.889246  2.347219e-03   
0.140271    18     -4.884790   0.003952 -4.880838  -4.889284  3.422018e-03   
0.196760    19     -4.884769   0.005438 -4.879331  -4.889231  4.891392e-03   
0.275997    20     -4.884561   0.007351 -4.877210  -4.888981  6.839393e-03   
0.387100    21     -4.883947   0.009772 -4.874175  -4.888314  9.332176e-03   
0.543100    22     -4.882393   0.012754 -4.869639  -4.886765  1.237248e-02   
0.761700    23     -4.879824   0.015543 -4.864281  -4.884289  1.519385e-02   
1.068500    24     -4.877047   0.017023 -4.860023  -4.881605  1.659090e-02   
1.498800    25     -4.874435   0.017331 -4.857103  -4.879057  1.670594e-02   
2.102400    26     -4.872012   0.017010 -4.855002  -4.876676  1.640668e-02   
2.949000    27     -4.869818   0.016421 -4.853397  -4.874510  1.616605e-02   
4.136600    28     -4.868014   0.015834 -4.852179  -4.872714  1.600531e-02   
5.802500    29     -4.866648   0.015655 -4.850993  -4.871335  1.603220e-02   
8.139200    30     -4.865504   0.016312 -4.849192  -4.870142  1.663618e-02   
11.417000   31     -4.864508   0.017925 -4.846583  -4.869069  1.819369e-02   
16.014700   32     -4.863932   0.020543 -4.843389  -4.868411  2.112059e-02   
22.464000   33     -4.864556   0.024467 -4.840089  -4.868997  2.577252e-02   
31.510500   34     -4.866880   0.031038 -4.835842  -4.871394  3.349129e-02   
44.200100   35     -4.870524   0.042389 -4.828135  -4.875212  4.680074e-02   
62.000000   36     -4.875458   0.060258 -4.815200  -4.880394  6.740087e-02   
85.775000   37     -4.881166   0.085838 -4.795328  -4.886346  9.456943e-02   
109.550000  38     -4.885607   0.112334 -4.773272  -4.890822  1.203877e-01   
133.325000  39     -4.889689   0.138166 -4.751522  -4.894761  1.450567e-01   
157.100000  40     -4.896302   0.171071 -4.725231  -4.901136  1.781925e-01   
180.875000  41     -4.906455   0.210273 -4.696182  -4.910899  2.218919e-01   
204.650000  42     -4.921596   0.255228 -4.666368  -4.925435  2.763867e-01   
228.425000  43     -4.942360   0.302454 -4.639906  -4.945499  3.355545e-01   
252.200000  44     -4.970556   0.351286 -4.619270  -4.973083  3.950239e-01   
275.975000  45     -5.010657   0.402232 -4.608425  -5.013221  4.524017e-01   
299.750000  46     -5.072874   0.457135 -4.615739  -5.076878  5.076118e-01   
323.525000  47     -5.161689   0.528857 -4.632831  -5.168271  5.739081e-01   
347.300000  48     -5.267386   0.634661 -4.632726  -5.277427  6.716781e-01   
371.075000  49     -5.379951   0.778143 -4.6018

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')